In [1]:
import pandas as pd

In [2]:
import warnings
warnings.simplefilter('ignore')

In [3]:
from data_processing import returns

In [4]:
returns

,Date,AGG,BIL,BND,EDV,IEF,IEI,SGOV,SHV,SHY,...,VOO,VTI,VTV,VUG,VWO,VXUS,XLE,XLF,XLG,XLK
0,2014-12-08,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2014-12-09,0.001547,-0.000219,0.000364,0.006597,0.003698,0.001965,NaN,0.000000,0.000355,...,-0.000792,0.000565,-0.002589,0.001816,-0.011263,-0.005607,0.010905,-0.002406,-0.001674,-0.000240
2,2014-12-10,0.001726,0.000000,0.002667,0.010867,0.004535,0.003023,NaN,0.000000,0.000828,...,-0.015685,-0.016376,-0.016521,-0.015649,-0.014364,-0.013492,-0.030821,-0.012862,-0.015024,-0.015115
3,2014-12-11,-0.000363,0.000000,-0.000967,0.004760,-0.001693,-0.001874,NaN,0.000091,-0.000473,...,0.004668,0.004593,0.005160,0.003974,-0.009296,-0.002450,-0.000663,0.002443,0.002838,0.004385
4,2014-12-12,0.003176,0.000219,0.003389,0.020990,0.008197,0.004162,NaN,0.000000,0.000946,...,-0.016235,-0.015716,-0.016832,-0.013035,-0.014963,-0.018212,-0.018563,-0.019496,-0.015352,-0.014795
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2508,2024-11-25,0.008997,0.000109,0.009230,0.037277,0.010791,0.005864,0.000099,0.000181,0.001951,...,0.003215,0.004890,0.006275,0.001658,0.000440,0.003284,-0.019739,0.006505,0.000407,-0.000043
2509,2024-11-26,-0.001520,0.000000,-0.001775,-0.004340,-0.002114,-0.000686,0.000199,0.000091,0.000000,...,0.005608,0.003658,0.000717,0.008352,-0.005283,-0.005073,-0.001363,0.001958,0.009565,0.005009
2510,2024-11-27,0.002740,0.000327,0.002735,0.009398,0.004025,0.002574,0.000199,0.000181,0.000609,...,-0.003187,-0.002842,-0.000441,-0.005783,0.003762,0.004441,-0.001050,0.001955,-0.005039,-0.013546
2511,2024-11-29,0.003947,0.000218,0.003682,0.013493,0.004220,0.002310,0.000397,0.000453,0.001216,...,0.005268,0.005567,0.003365,0.008405,0.004630,0.010480,0.004310,0.001561,0.008306,0.009284


In [5]:
# change to long format to prepare for prophet modeling
long_format = returns.melt(id_vars=['Date'],
                           var_name = 'Stock',
                           value_name = 'Return')


In [6]:
long_format

,Date,Stock,Return
0,2014-12-08,AGG,NaN
1,2014-12-09,AGG,0.001547
2,2014-12-10,AGG,0.001726
3,2014-12-11,AGG,-0.000363
4,2014-12-12,AGG,0.003176
...,...,...,...
118106,2024-11-25,XLK,-0.000043
118107,2024-11-26,XLK,0.005009
118108,2024-11-27,XLK,-0.013546
118109,2024-11-29,XLK,0.009284


In [7]:
# rename to match compatibility with prophet 
long_format.rename(columns= {'Date':'ds','Return':'y'},inplace=True)

In [8]:
long_format

,ds,Stock,y
0,2014-12-08,AGG,NaN
1,2014-12-09,AGG,0.001547
2,2014-12-10,AGG,0.001726
3,2014-12-11,AGG,-0.000363
4,2014-12-12,AGG,0.003176
...,...,...,...
118106,2024-11-25,XLK,-0.000043
118107,2024-11-26,XLK,0.005009
118108,2024-11-27,XLK,-0.013546
118109,2024-11-29,XLK,0.009284


In [9]:
# place each stock in groups
stock_groups = {stock:group[['ds','y']] for stock, group in long_format.groupby('Stock')}

In [10]:
stock_groups['AGG']

,ds,y
0,2014-12-08,NaN
1,2014-12-09,0.001547
2,2014-12-10,0.001726
3,2014-12-11,-0.000363
4,2014-12-12,0.003176
...,...,...
2508,2024-11-25,0.008997
2509,2024-11-26,-0.001520
2510,2024-11-27,0.002740
2511,2024-11-29,0.003947


In [11]:
from prophet import Prophet

In [18]:
# loop through stocks to get future predictions on each stock
def forecast_returns(stock_groups):
    forecasts = {}
    
    for stock, data in stock_groups.items():
        model = Prophet()
        model.fit(data)
        futureOneYear = model.make_future_dataframe(periods=365)  # Forecasting 1 year ahead
        forecast = model.predict(futureOneYear)
    
        forecasts[stock] = forecast[['ds', 'yhat']] 
    
    return forecasts
    

In [20]:
forecasts = forecast_returns(stock_groups)

15:26:39 - cmdstanpy - INFO - Chain [1] start processing
15:26:40 - cmdstanpy - INFO - Chain [1] done processing
15:26:40 - cmdstanpy - INFO - Chain [1] start processing
15:26:40 - cmdstanpy - INFO - Chain [1] done processing
15:26:41 - cmdstanpy - INFO - Chain [1] start processing
15:26:41 - cmdstanpy - INFO - Chain [1] done processing
15:26:41 - cmdstanpy - INFO - Chain [1] start processing
15:26:41 - cmdstanpy - INFO - Chain [1] done processing
15:26:42 - cmdstanpy - INFO - Chain [1] start processing
15:26:42 - cmdstanpy - INFO - Chain [1] done processing
15:26:42 - cmdstanpy - INFO - Chain [1] start processing
15:26:42 - cmdstanpy - INFO - Chain [1] done processing
15:26:43 - cmdstanpy - INFO - Chain [1] start processing
15:26:43 - cmdstanpy - INFO - Chain [1] done processing
15:26:44 - cmdstanpy - INFO - Chain [1] start processing
15:26:44 - cmdstanpy - INFO - Chain [1] done processing
15:26:44 - cmdstanpy - INFO - Chain [1] start processing
15:26:44 - cmdstanpy - INFO - Chain [1]

In [21]:
forecasts['AGG']

,ds,yhat
0,2014-12-08,-0.000116
1,2014-12-09,0.000366
2,2014-12-10,0.000579
3,2014-12-11,0.000247
4,2014-12-12,0.000311
...,...,...
2873,2025-11-28,0.000928
2874,2025-11-29,0.001690
2875,2025-11-30,0.001683
2876,2025-12-01,0.000282


In [25]:
# forecast sent to CSV for optimization
forecast_df = pd.concat(forecasts.values(), keys = forecasts.keys())

In [26]:
forecast_df.to_csv('forecasted_returns.csv')